In [2]:
# notebooks/01_baseline_test.ipynb

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import sys
import os

c:\Users\lsoo6\Desktop\GrowIT\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import config

In [4]:
print(f"--- 1단계: 베이스라인 테스트 시작 ---")
print(f"로드할 베이스 모델 ID: {config.BASE_MODEL_ID}")

--- 1단계: 베이스라인 테스트 시작 ---
로드할 베이스 모델 ID: meta-llama/Llama-3.1-8B-Instruct


In [5]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 
)

In [6]:
try:
    model = AutoModelForCausalLM.from_pretrained(
        config.BASE_MODEL_ID,
        quantization_config=quantization_config, # 4비트 설정 적용
        device_map="auto",
        torch_dtype=torch.bfloat16
    )
    
    # --- 2. 토크나이저 로드 ---
    tokenizer = AutoTokenizer.from_pretrained(config.BASE_MODEL_ID)

    print("\n--- Llama 3.1 8B 모델 및 토크나이저 로드 성공 ---")

except Exception as e:
    print(f"\n--- 모델 로드 실패 ---")
    print(f"오류: {e}")
    # (여기서 멈춤)

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 4/4 [04:31<00:00, 67.96s/it]



--- Llama 3.1 8B 모델 및 토크나이저 로드 성공 ---


In [8]:
if 'model' in locals():
    prompt = "오늘 축구했어"

    messages = [
        {"role": "system", "content": "You are a helpful and empathetic AI assistant."},
        {"role": "user", "content": prompt}
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        input_ids,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
    )

    response_ids = outputs[0][input_ids.shape[-1]:]
    result_text = tokenizer.decode(response_ids, skip_special_tokens=True)

    print(f"\n--- 🕵️ 베이스라인 테스트 결과 ---")
    print(f"User: {prompt}")
    print(f"AI (Llama 3.1 날것): {result_text}")
    print("-----------------------------------")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



--- 🕵️ 베이스라인 테스트 결과 ---
User: 오늘 축구했어
AI (Llama 3.1 날것): 축구는 정말 재미있는 운동이죠. 어떻게 경기가 흘러갔는지 궁금하네요. 승리했는지, 패배했는지, 혹은 무승부였는지 알려주세요!
-----------------------------------
